In [2]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
datagen_training = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [4]:
datagen_default = tf.keras.preprocessing.image.ImageDataGenerator(
)

In [5]:
BATCH_SIZE = 16
train_generator = datagen_training.flow_from_directory(
    "hand_data/train",
    target_size=(64, 64),
    batch_size=BATCH_SIZE,
    class_mode="binary",
)

val_generator = datagen_training.flow_from_directory(
    "hand_data/val",
    target_size=(64, 64),
    batch_size=BATCH_SIZE,
    class_mode="binary",
)

def new_gen(gen):
    for i, j in gen:
        yield i, tf.expand_dims(j, -1)
        
train_generator = new_gen(train_generator)
val_generator = new_gen(val_generator)
# train_generator = train_generator.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))
# val_generator = val_generator.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))
x_train = []
y_train = []
x_val = []
y_val = []
for i, j in train_generator:
    print("-------")
#     x_train.append(i)
#     y_train.append(j)
# for i, j in val_generator:
#     x_val = np.append(x_val, i)
#     y_val = np.append(y_val, j)
# train_generator = detagen_default.flow(x_train, y_train)
# val_generator = detagen_default.flow(x_val, y_val)

Found 89 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------


KeyboardInterrupt: 

In [ ]:
# Lets have a look at some of our images
images, labels = train_generator.next()
print(images.shape, labels.shape)

fig = plt.figure(figsize=(20,10))
fig.subplots_adjust(wspace=0.2, hspace=0.4)

# Lets show the first 32 images of a batch
for i, img in enumerate(images[:32]):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(img)
    image_idx = np.argmax(labels[i])

In [ ]:
tf.keras.backend.clear_session()

inputs = tf.keras.layers.Input(shape=(64, 64, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(inputs)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(rate=0.5)(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation ='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation ='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
x = tf.keras.layers.Dropout(rate=0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(rate=0.5)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
# tf.keras.backend.clear_session()

# inputs = tf.keras.layers.Input(shape=(128, 128, 3))
# x = tf.keras.applications.MobileNetV2(
#     include_top=False,
#     input_tensor=inputs,
#     input_shape=(128, 128, 3),
#     pooling="avg"
# )

# for l in x.layers:
#     l.trainable = True

# x = tf.keras.layers.Dense(256, activation="relu")(x.output)
# x = tf.keras.layers.Dropout(0.8)(x)
# outputs = tf.keras.layers.Dense(2, activation="sigmoid")(x)

# model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# num_train = len(train_generator.filenames)
# num_valid = len(val_generator.filenames)

# Reduce learning rate when loss doesn't improve after n epochs
scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=1e-8, verbose=1)

# Stop early if model doesn't improve after n epochs
early_stopper = EarlyStopping(monitor='val_loss', patience=12,
                              verbose=0, restore_best_weights=True)

# Train the model
history = model.fit(train_generator,
                    epochs=100,
                    verbose=1,
                    callbacks=[scheduler, early_stopper],
                    validation_data=val_generator)

In [ ]:
# tf.keras.backend.clear_session()

# inputs = tf.keras.layers.Input(shape=(64, 64, 3))
# x = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(inputs)
# x = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(x)
# x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
# x = tf.keras.layers.Dropout(rate=0.5)(x)

# x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation ='relu')(x)
# x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation ='relu')(x)
# x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
# x = tf.keras.layers.Dropout(rate=0.25)(x)

# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
# x = tf.keras.layers.Dropout(rate=0.5)(x)
# outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

# model = tf.keras.Model(inputs=inputs, outputs=outputs)
# model.summary()